In [1]:
# need this for interactive plots in jupyter notebooks
%matplotlib widget
import os # for file path manipulations
import yaml # for reading yaml files (i.e. config file?)
import time # for timing code execution and sleep functions
import numpy as np
import sys # access and modify python runtime env. (sys.path)
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import socket # for network comm
import threading # for running concurrent threads
from pathlib import Path

TCP_IP = "192.168.14.14"
TCP_PORT = 65432
BUFFER_SIZE = 1024

sys.path.append('../../') # change this depending on final location of this script
from geecs_python_api.controls.interface import GeecsDatabase
from geecs_python_api.controls.devices.geecs_device import GeecsDevice


path config ..\..\..\..\..\..\user data\Configurations.INI


In [ ]:
# TCP Server - monitors multiple devices and broadcasts interlock flags

# Load device config from YAML file
with open('software_interlock.yaml', 'r') as f:
    yml_data = yaml.safe_load(f)

experiment = yml_data['experiment'] 
devices_config = yml_data['devices']

GeecsDevice.exp_info = GeecsDatabase.collect_exp_info(experiment)

# Dictionary to store interlock flags for each device
interlock_flags = {}
device_cameras = {}
flags_lock = threading.Lock()
server_running = True

# Initialize devices
for dev_cfg in devices_config:
    if dev_cfg.get('active', False):
        dev_name = dev_cfg['name']
        device_cameras[dev_name] = {
            'camera': GeecsDevice(dev_name),
            'variable': dev_cfg.get('variable', 'MaxCounts'),
            'threshold': dev_cfg.get('threshold', 1000)
        }
        device_cameras[dev_name]['camera'].subscribe_var_values([dev_cfg.get('variable', 'MaxCounts')])
        interlock_flags[dev_name] = False
        print(f"Initialized: {dev_name}")

def interlock_monitor(dev_name, dev_info):
    """Background thread that monitors a single device and updates its interlock flag"""
    camera = dev_info['camera']
    variable = dev_info['variable']
    threshold = dev_info['threshold']
    
    while server_running:
        try:
            current_value = camera.get(variable)
            
            if current_value is not None:
                with flags_lock:
                    if current_value > threshold:
                        if not interlock_flags[dev_name]:  # Only print on state change
                            print(f"INTERLOCK TRIGGERED: {dev_name} {variable}={current_value} > {threshold}")
                        interlock_flags[dev_name] = True
                    else:
                        interlock_flags[dev_name] = False
        except Exception as e:
            print(f"Error monitoring {dev_name}: {e}")
        
        time.sleep(0.5)

def handle_interlock_client(conn, addr):
    """Send interlock status updates to connected clients"""
    print(f"Client connected: {addr}")
    try:
        while server_running:
            with flags_lock:
                status_lines = []
                for dev_name, flag in interlock_flags.items():
                    status = "INTERLOCK_ACTIVE" if flag else "OK"
                    status_lines.append(f"{dev_name}: {status}")
                message = " | ".join(status_lines) + "\n"
            
            conn.sendall(message.encode('utf-8'))
            time.sleep(0.5)
    except (BrokenPipeError, ConnectionResetError):
        print(f"Client disconnected: {addr}")
    finally:
        conn.close()

def run_interlock_server(host="127.0.0.1", port=5001):
    """Run TCP server for interlock monitoring"""
    # Start monitoring threads for each device
    for dev_name, dev_info in device_cameras.items():
        monitor_thread = threading.Thread(target=interlock_monitor, args=(dev_name, dev_info), daemon=True)
        monitor_thread.start()
    
    # Start TCP server
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.bind((host, port))
        s.listen()
        print(f"Interlock server listening on {host}:{port}")
        print(f"Monitoring {len(device_cameras)} device(s)")
        
        while server_running:
            try:
                s.settimeout(1.0)
                conn, addr = s.accept()
                client_thread = threading.Thread(target=handle_interlock_client, args=(conn, addr), daemon=True)
                client_thread.start()
            except socket.timeout:
                continue

# Run the server
run_interlock_server()

INTERLOCK TRIGGERED: MaxCounts=146.0 exceeds threshold 140
Initialized: CAM-PL1-TapeDrivePointing
Interlock server listening on 127.0.0.1:5001
Monitoring 1 device(s)
INTERLOCK TRIGGERED: CAM-PL1-TapeDrivePointing MaxCounts=149.0 > 140
Initialized: CAM-PL1-TapeDrivePointing
Interlock server listening on 127.0.0.1:5001
Monitoring 1 device(s)
INTERLOCK TRIGGERED: CAM-PL1-TapeDrivePointing MaxCounts=149.0 > 140
INTERLOCK TRIGGERED: MaxCounts=143.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=143.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=151.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=151.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=147.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=147.0 exceeds threshold 140
INTERLOCK TRIGGERED: CAM-PL1-TapeDrivePointing MaxCounts=141.0 > 140
INTERLOCK TRIGGERED: MaxCounts=153.0 exceeds threshold 140
INTERLOCK TRIGGERED: CAM-PL1-TapeDrivePointing MaxCounts=141.0 > 140
INTERLOCK TRIGGERED: MaxCounts=153.0 e

Exception in thread Thread-816 (handle_interlock_client):
Traceback (most recent call last):
  File "c:\Users\loasis\AppData\Local\anaconda3\envs\geecs\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Users\loasis\AppData\Local\anaconda3\envs\geecs\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\loasis\AppData\Local\Temp\ipykernel_17808\4205843681.py", line 66, in handle_interlock_client
ConnectionAbortedError: [WinError 10053] An established connection was aborted by the software in your host machine


INTERLOCK TRIGGERED: CAM-PL1-TapeDrivePointing MaxCounts=147.0 > 140
INTERLOCK TRIGGERED: MaxCounts=151.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=151.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=142.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=142.0 exceeds threshold 140


KeyboardInterrupt: 

INTERLOCK TRIGGERED: MaxCounts=143.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=145.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=145.0 exceeds threshold 140
INTERLOCK TRIGGERED: CAM-PL1-TapeDrivePointing MaxCounts=141.0 > 140
INTERLOCK TRIGGERED: MaxCounts=143.0 exceeds threshold 140
INTERLOCK TRIGGERED: CAM-PL1-TapeDrivePointing MaxCounts=141.0 > 140
INTERLOCK TRIGGERED: MaxCounts=143.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=142.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=142.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=143.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=143.0 exceeds threshold 140
INTERLOCK TRIGGERED: CAM-PL1-TapeDrivePointing MaxCounts=152.0 > 140
INTERLOCK TRIGGERED: CAM-PL1-TapeDrivePointing MaxCounts=152.0 > 140
INTERLOCK TRIGGERED: MaxCounts=143.0 exceeds threshold 140
INTERLOCK TRIGGERED: MaxCounts=143.0 exceeds threshold 140
INTERLOCK TRIGGERED: CAM-PL1-TapeDrivePointing MaxCounts=151.0 > 140
INTERL

In [ ]:
def live_plot(camera, threshold=1000):    
    fig, ax = plt.subplots()
    x, y = [] , []
    line, = ax.plot([],[], 'o-',color='blue')
    start_time = time.time()
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Max Counts')
    ax.grid()
    try:

        while True:
            current_time = time.time() - start_time
            max_counts = camera.get('MaxCounts')

            x.append(current_time)
            y.append(max_counts)

            if len(x) > 50:
                x.pop(0)
                y.pop(0)

            line.set_data(x, y)
            ax.relim()
            ax.autoscale_view()

            clear_output(wait=True)
            display(fig)

            time.sleep(0.01)

            if max_counts > threshold:
                print(f"Max Counts exceeded threshold! Current value: {max_counts}")
                break
    except KeyboardInterrupt:
        print("Live Plot Stopped")

In [ ]:
# load and read the interlock yaml file
with open('software_interlock.yaml', 'r') as f:
    yml_data = yaml.safe_load(f)

experiment = yml_data['experiment']
devices = yml_data['devices']

INTERLOCK_VAR = False

GeecsDevice.exp_info = GeecsDatabase.collect_exp_info(experiment)
camera = {}
for device in devices:
    if device.get('active', True):
        camera[device.get('name')] = GeecsDevice(device.get('name'))
        camera[device.get('name')].subscribe_var_values(['MaxCounts'])
        time.sleep(1)
        print(camera[device.get('name')].state)
max_counts = camera.get('MaxCounts')
print(max_counts)

while True:
    max_counts = camera.get('MaxCounts')
    time.sleep(1)
    print(max_counts)
    if max_counts > device.get('threshold'):
        print(f"{device.get('name')} Max Counts exceeded threshold! Current value: {max_counts}")
        INTERLOCK_VAR = True
        # set interlock variable to ok or NOT OK depending on whether the variable exceeds the threshold or not

        


In [ ]:
# load and read the interlock yaml file
with open('software_interlock.yaml', 'r') as f:
    yml_data = yaml.safe_load(f)

experiment = yml_data['experiment']
devices = yml_data['devices']

GeecsDevice.exp_info = GeecsDatabase.collect_exp_info(experiment)

alert_queue = []
queue_lock = threading.Lock()

def _watch_device(device):
    """
    Watch a single device variable and add an alert to the queue if it exceeds the variable threshold.
    """
    if not device.get('active', False):
        return
    camera = GeecsDevice(device.get('name'))
    camera.subscribe_var_values(device.get('variable'))
    exposure = camera.get('exposure')
    print(f"{device.get('name')} exposure: {exposure}")

    variable_name = device.get('variable')
    threshold = device.get('threshold')

    while True:
        try:
            current_value = camera.get(variable_name)
        except Exception as exc:
            with queue_lock:
                alert_queue.append(f"ERROR {device['name']} {variable_name}: {exc}")
            break

        if current_value is None:
            time.sleep(0.05)
            continue

        if current_value > threshold:
            message = (
                f"ALERT {device['name']} {variable_name}={current_value} exceeded threshold {threshold}"
            )
            with queue_lock:
                alert_queue.append(message)
            break

        time.sleep(0.05)

def _handle_client(conn, addr):
    print(f"Connected by {addr}")
    with conn:
        conn.settimeout(0.2)
        while True:
            try:
                data = conn.recv(BUFFER_SIZE)
            except socket.timeout:
                data = b""
            if data:
                message = data.decode('utf-8', errors='replace').strip()
                if message.lower().startswith('start'):
                    print("Starting interlock monitoring...")
                    for device in devices:
                        thread = threading.Thread(target=_watch_device, args=(device,), daemon=True)
                        thread.start()
                    conn.sendall(b"Monitoring started\n")
                else:
                    conn.sendall(b"Commands: START\n")

            with queue_lock:
                while alert_queue:
                    alert = alert_queue.pop(0)
                    conn.sendall((alert + "\n").encode('utf-8'))
    print(f"Disconnected: {addr}")

def run_tcp_server(host=TCP_IP, port=TCP_PORT):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        s.bind((host, port))
        s.listen()
        print(f"Server listening on {host}:{port}")
        while True:
            conn, addr = s.accept()
            thread = threading.Thread(target=_handle_client, args=(conn, addr), daemon=True)
            thread.start()

run_tcp_server()

In [ ]:
# set the experiment info for the GeecsDevice class
GeecsDevice.exp_info = GeecsDatabase.collect_exp_info("Bella")

# define the device name 
camera = GeecsDevice('CAM-PL1-TapeDrivePointing')
exposure = camera.get('exposure')
print(exposure)

print(camera.state)

# example of writing to a device parameter
camera.set('exposure', 0.1)

print(camera.get('MaxCounts'))
